# MCP Architecture

In [13]:
from IPython.display import display, SVG

## Architecture Entities (Host, Client and Server)

- In MCP architecture, there are mainly 2 entities
    1. Host
    2. MCP Server
- Host is basically the ChatBot like Claude Desktop or ChatGPT or your Custom ChatBot with which User interacts directly in natural language. And in the backend this host is connected  to an LLM like Sonnet or GPT.
- Server can be a MCP server of GitHub or Slack or Google Drive.
- All the communication happens between the Host and the Server.

**Simplified version of MCP Architecture**
Example: Suppose you type a prompt on the ChatBot (the Host): “Are there any new commits in my GitHub repo ‘rag’?” The ChatBot forwards this prompt to the LLM. The LLM determines that it doesn’t have the answer directly but knows that the GitHub server might. It informs the ChatBot accordingly. The ChatBot then connects to the GitHub server and retrieves the necessary data. After receiving the response, the ChatBot sends it to the LLM. With this information, the LLM formulates the final answer and sends it back to the Host, which then displays the response to the user.

Conversation Format:
* User: Are there any new commits in my GitHub repo “rag”?
* ChatBot (Host): Let me check that for you. [Sends the prompt to the LLM]
* LLM: I don’t have that information directly, but the GitHub server might have the answer.
* ChatBot (Host): Got it. I’ll contact the GitHub server. [Connects to GitHub and retrieves the data]
* ChatBot (Host): Here’s the response from the GitHub server. [Sends it to the LLM]
* LLM: Great, now I have enough information to answer your question. The repo “rag” has 2 new commits since your last update.
* ChatBot (Host): The repo “rag” has 2 new commits since your last update.

In [17]:
display(SVG(filename='mcp_architecture_1.drawio.svg'))

A small detail to note - the Host cannot directly communicate with the MCP Server. To do that, it uses an intermediary called the MCP Client. So now, the conversation would look like this:
* User: Are there any new commits in my GitHub repo “rag”?
* ChatBot (Host): Let me check that for you. [Sends the prompt to the LLM]
* LLM: I don’t have that information directly, but the GitHub server might have the answer.
* ChatBot (Host): Got it. I’ll contact the GitHub server.
* ChatBot (Host): Creates a high-level request — “Find the recent commits in repo rag” — and sends it to the MCP Client.
* MCP Client: Converts the request into an MCP-compatible format and forwards it to the MCP Server.
* MCP Server: Since the request is in an MCP-compatible format, it can process it. It retrieves the required data and returns the response in MCP format.
* MCP Client: Receives the response and translates it into a format that the Host can understand.
* ChatBot (Host): Here’s the response from the GitHub server. [Sends it to the LLM]
* LLM: Perfect, now I have enough information to answer your question. The repo “rag” has 2 new commits since your last update.
* ChatBot (Host): The repo “rag” has 2 new commits since your last update.

In [19]:
display(SVG(filename='mcp_architecture_2.drawio.svg'))

Note: The relationship between an MCP Client and MCP Server is 1-1. GitHub MCP Client can only communicate with GitHub MCP Server. So if the Host wants to connect with Google Drive MCP Server, it has to connect with Google Drive MCP Client.

In [22]:
display(SVG(filename='mcp_architecture_3.drawio.svg'))

## MCP Primitives

Primitives represent the core functionalities that a server can provide to the Host. There are three main types:

1. **Tools** – Dynamic capabilities that allow the Host to gain knowledge by performing specific actions.
2. **Resources** – Static information or knowledge that can be directly retrieved.
3. **Prompts** – Predefined prompt templates or instructions offered by the server to help guide or refine the LLM’s responses.

### Example (Using GitHub MCP Server)

* **Tools:** Actions like finding the number of commits on a repo, checking active issues, listing GitHub repositories, or creating a new repo named "rag".
* **Resource:** Fetching static information such as the GitHub account name.
* **Prompts:** Templates that help the LLM produce clearer, structured outputs before interacting with tools.

#### Example Scenario:

**User:** “Create an issue in GitHub for repo RAG, login button does not work.”
**Host → LLM:** Sends the request.
**LLM:** Recognizes the GitHub tool and calls `/create-issue` with:

```
Title: Login Button Issues
Description: Login button does not work while the user tries to log in
```

This issue report is too vague.

Here, the **Prompts** primitive improves the outcome. The server provides a predefined prompt template to help the LLM generate more complete, high-quality issue reports.

**Prompt Template Example:**

```json
{
  "name": "issue_report_prompt",
  "description": "Write clear, detailed GitHub issues",
  "messages": [{
    "role": "system",
    "content": "Always include: Title, Steps to Reproduce, Expected, Actual, Environment"
  }]
}
```

**Refined Example:**

**User:** “Create an issue in GitHub for repo RAG, login button does not work.”

**Host → LLM:** Sends the request.

**LLM (using the prompt):** Calls `/create-issue` with:

**Title:** Bug - Login Button Not Working

**Steps to Reproduce:**

1. Open the login page
2. Enter valid credentials
3. Click the Login button

**Expected Behavior:**
User should be logged in and redirected to the dashboard.

**Actual Behavior:**
Nothing happens when clicking the Login button.

**Environment:**
Chrome 121, macOS 14.2


### Standard Operations

* **Tools**
    * `tools/list` $\rightarrow$ Client asks the Server: "*What tools do you provide?*"
    * `tools/call` $\rightarrow$ Client tells the Server: "*Please run this tool with these arguments.*"
* **Resources**
    * `resources/list` $\rightarrow$ Client asks: "*What resources are available?*"
    * `resources/read` $\rightarrow$ Client says: "*Give me the content of this resource.*"
    * `resources/subscribe / unsubscribe` $\rightarrow$ Client subscribes or unsubscribes from updates.
* **Prompts**
    * `prompts/list` $\rightarrow$ Client asks: "*What prompt templates do you provide?*"
    * `prompts/get` $\rightarrow$ Client fetches a specific prompt template.

In [24]:
display(SVG(filename='mcp_architecture_4.drawio.svg'))

## Data Layer

- MCP Data Layer is the topic about **how MCP Client sends the request to MCP Server**. The answer to this HOW is JSON RPC 2.0. For this first make yourself familier with RPC.
- RPC, to understand, is like REST or SOAP, an architectural paradigm(technically a protocol though) that is used to have a communication between 2 machines.
- Very famous way to do this is REST, but RPC is used especially used in distributed systems.
- There are mainly 2 frameworks of RPC protocol, gRPC and JSON RPC. 
- [Here](https://www.youtube.com/watch?v=GaDo88eFzZ4) is a youtube video which explains rpc, json-rpc and grpc in go language.
- But one thing is for sure, whether it is gRPC or JSON RPC 2.0, they use a protocol to send that data from one computer to another computer. This sending happens through a protocol. In case of gRPC, this protocol is HTTP/2 but in case of JSON RPC 2.0, this protocol can be anything. **This is an important point**.
- You need to know more about RPC, gRPC, JSON-RPC and where and why these things are used. Once you have an understanding, the Data and Transport Layer concept of MCP Architecture will be very easy to understand. So no need to note anything on specifics of how JSON RPC 2.0 request looks like in terms of MCP Client and MCP Server communication. Still if you want to see, [here](https://modelcontextprotocol.io/docs/learn/architecture) is the documentation.
- After knowing about how RPC works, specific to MCP communication you need to know below things.
- In case of MCP Client Server communication, usually there are 3 things to know
    - the method call can be tools/list, tools/call
    - the method call can be resources/read
    - We can do batching(because this is possible in JSON rpc)
    - We can use notifications on change in tools(if there is a change in someting, then the server will notify). Also in case of notification, you don't need id(in other requests, you need id, because you are expecting response, but as in case of notification, there is no response, so no id is required. I think this is a RPC thing, but need to check.)
        - notification vs request/response
- **Question** Why did Anthropic team choose JSON RPC protocol to communicate between client and server? Why not REST API? What's wrong with that?
    1. **JSON RPC is transport agnostic.** It does not mandate any specific transport protocol like HTTP/SMTP/FTP etc anything works. This point alone is the entire MCP Transport Layer.
    2. JSON RPC supports 2 way communication, but REST only supports 1 way communication
    3. JSON RPC supports batching.
    4. JSON RPC supports notifications.

## Transport Layer

- To send a request, MCP Client has to basically send some data(which is in JSON format and as per MCP standard) to the MCP Server. To send response MCP Server has to send some data(which is also JSON format and as per MCP standard) to MCP Client. Now this sending must be happening via some protocol. MCP Transport layer basically defines that.
- MCP Client will always remain on the system that you are working on
- But MCP Server may remain in your system or may be remote. So MCP Server can be of 2 types
    1. Local Server: File System MCP Server
    2. Remote Server: GitHub MCP Server
- In case of Local Server, STDIO is used for transport data between server and client.
- In case of Remote Server, HTTP/SSE is used for transport data between server and client.
- In the MCP Data Layer, I wrote, JSON RPC 2.0 does not depends on any specific protocol. Because this is the case, we can have either a local server or a remote server. And tomorrow in transport layer, say some other protocol comes it, MCP will support that, because at the end of the day, it does not depend on any protocol in the first place to transport the JSON data. AND that is one of the main reason why Anthropic team choose JSON RPC 2.0 for the mode of transport.

**To understand all these with full clarity, you need to understand**
- HTTP(The JSON data is sent as a payload in a HTTP request and response for communication.)
- SSE (Server Sent Events - Basically I wrote that, JSON RPC supports 2 way communication, SSE helps for that. SSE is an extension of HTTP. HOW? learn that)
- STDIO (Running a python file to get input from user and show output to user in terminal is STDIO. HOW? learn that)

In [25]:
display(SVG(filename='mcp_architecture_5.drawio.svg'))